In [ ]:
import pandas as pd
import sqlite3

Создадим подключение к базе данных с помощью библиотеки sqlite3

In [ ]:
col_names = ['datetime', 'user']
connect = sqlite3.connect('../data/checking-logs.sqlite')

In [ ]:
query = 'PRAGMA table_info(test);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


Получим первые 10 строк просмотров страниц таблицы, чтобы проверить, как выглядит таблица

In [ ]:
query = "SELECT * FROM test LIMIT 10"
pd.io.sql.read_sql(query, connect)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


Посмотрим, что за данные в таблице deadlines

In [ ]:
query = 'PRAGMA table_info(deadlines);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [ ]:
query = "SELECT * FROM deadlines LIMIT 10"
pd.io.sql.read_sql(query, connect)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


Видим, что дата в этой таблице указана количества секунд с 1 января 1970 года. Поэтому нам надо будет привести даты к одному виду.

Найдем среди всех пользователей минимальное значение дельты между первой фиксацией пользователя и крайним сроком соответствующей лаборатории, используя только один запрос

CAST позволяет преобразовать данные X в данные другого типа Y

JulianDay количество дней с полудня 24 ноября 4714 года до н. э. https://oracleplsql.ru/julianday-sqlite.html

DATETIME https://bravikov.wordpress.com/2013/08/26/функции-даты-и-временем-sqlite/

unixepoch вычисляет количество секунд с 1 января 1970 года

LEFT JOIN создает левое внешнее соединение. Выбираются все записи первой (левой) таблицы, даже если они не соответствуют записям во второй (правой) таблице.

<> Не равно

In [ ]:
query = '''
SELECT uid,
       MIN(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_min = pd.io.sql.read_sql(query, connect)
df_min

,uid,delta
0,user_25,2


Сделаем то же самое, но для максимума

In [ ]:
query = '''
SELECT uid,
       MAX(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_max = pd.io.sql.read_sql(query, connect)
df_max

,uid,delta
0,user_30,202


Сделаем то же самое для среднего значения. Фрейм данных не должен включать столбец uid.

In [ ]:
query = '''
SELECT AVG(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_avg = pd.io.sql.read_sql(query, connect)
df_avg

,delta
0,89.125


Создадим таблицу со столбцами: uid, avg_diff, pageviews согласно заданию

In [ ]:
query = '''
SELECT test.uid,
       AVG(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) - 
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta, 
       pageviews
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
LEFT JOIN (SELECT uid, count(*) AS pageviews
           FROM pageviews
           GROUP BY uid) AS views ON test.uid=views.uid
WHERE labname <> 'project1'
GROUP BY test.uid
'''
views_diff = pd.io.sql.read_sql(query, connect)
views_diff

,uid,delta,pageviews
0,user_1,64.400000,28
1,user_10,74.800000,89
2,user_14,159.000000,143
3,user_17,61.600000,47
4,user_18,5.666667,3
5,user_19,98.750000,16
6,user_21,95.500000,10
7,user_25,92.600000,179
8,user_28,86.400000,149
9,user_3,105.400000,317


Рассчитаем коэффициент корреляции между количеством просмотров страниц и разницей между первой фиксацией и дедлайном

In [ ]:
views_diff.corr()

,delta,pageviews
delta,1.000000,0.279736
pageviews,0.279736,1.000000


Закроем соединение

In [ ]:
connect.close()